# Import and definitions

In [ ]:
import glob
import os
import sys
import time
import uuid
import yaml
import zmq
from numpy import pi
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import qutip as qt

# Specify initial pulse and bounds

In [ ]:
from c3po.utils.envelopes import flattop

def my_flattop(t, idx, guess):
    t_up = guess[idx['t_up']]
    t_down = guess[idx['t_down']]
    return flattop(t, t_up, t_down)

handmade_pulse = {
        'control1': {
            'carrier1': {
                'freq': 6.234e9*2*pi,
                'pulses': {
                    'pulse': {
                        'params': {
                            'amp': 25e6*2*pi,
                            't_up': 5e-9,
                            't_down': 25e-9,
                            'xy_angle': 0,
                            'freq_offset': 0e6*2*pi
                            },
                        'func': my_flattop
                        }
                    }
                }
            }
        }

pulse_bounds = {
        'control1': {
            'carrier1': {
                'pulses': {
                    'pulse': {
                        'params': {
                            'amp': [1e3*2*pi, 1e9*2*pi],
                            't_up': [2e-9, 98e-9],
                            't_down': [2e-9, 98e-9],
#                            'xy_angle': [-pi, pi],
                            'freq_offset': [-1e9*2*pi, 1e9*2*pi]
                            }
                        }
                    }
                }
            }
        }

initial_spread = {
        'control1': {
            'carrier1': {
                'pulses': {
                    'pulse': {
                        'params': {
                            'amp': 5e6*2*pi,
                            't_up': 5e-9,
                            't_down': 5e-9,
#                            'xy_angle': [-pi, pi],
                            'freq_offset': 20e6*2*pi
                            }
                        }
                    }
                }
            }
        }

# Evaluation function

Provide a function, specific to your experimental setup, that takes a several sets of parameters and provides a figure of merit for each.

In [ ]:
def evaluate_pulse(gate, samples):
    global search_id
    infidelities = []
    for q in samples:
        pulse_id = str(uuid.uuid4())
        clear_output(wait=True)
        gate.print_pulse(q)
        IQ = gate.get_IQ(q)
        T_final = gate.T_final
        ts = np.linspace(0, T_final, T_final*1e9)
        Is = np.array(IQ['control1']['carrier1']['I'])
        Qs = np.array(IQ['control1']['carrier1']['Q'])
        non0_mask = np.nonzero(Is * Qs)
               
        plt.cla()
        plt.plot(ts / 1e-9, Is)
        plt.plot(ts / 1e-9, Qs)
        plt.legend(("I", "Q"))
        plt.ylabel('I/Q')
        plt.xlabel('Time[ns]')
        plt.tick_params('both',direction='in')
        plt.tick_params('both', direction='in')
        plt.draw()
        fig.canvas.flush_events()
        pulse = {}
        pulse['I'] = list(Is[non0_mask])  
        pulse['Q'] = list(Qs[non0_mask])
        pulse['omegas'] = [IQ['control1']['carrier1']['omega']]
        pulse['carrier_amp'] = IQ['control1']['carrier1']['amp']
        request = {
                'search_id': search_id,
                'pulse_id': pulse_id,
                'pulse': pulse,
                'fidelity': 0,
                'do_stop': False
                }
        
        socketreq.send_json(request)
        receive_message = socketreq.recv_string()        
        reply = socketrep.recv_json()
        socketrep.send_string(search_id)    
        infidelities.append(1-reply['fidelity'])
    return infidelities

# Import C3PO

In [ ]:
from c3po.main.gate import Gate as gt
from c3po.fidelity.measurement import Experiment as exp_backend

In [ ]:
X_gate = gt('qubit_1', qt.sigmax())
X_gate.set_parameters('initial', handmade_pulse)
X_gate.set_bounds(pulse_bounds)

fridge = exp_backend(evaluate_pulse)

In [ ]:
X_gate.idxes

# Communication setup
Replace '127.0.0.1' with the IP of the LabView machine. Hostnames might confuse the queue interface.

In [ ]:
calibration_daemon_experiment_URI = "tcp://127.0.0.1:5559"
calibration_daemon_searcher_URI = "tcp://127.0.0.1:5560"

   
rcvtimeout = 30000
search_id = str(uuid.uuid4())
print(f"I am the calibration searcher.\nMy ID for this run is {search_id}\n")

# Start communication
print(f"Connecting to client {calibration_daemon_experiment_URI} ... ", flush=True, end='')
context = zmq.Context()
socketrep = context.socket(zmq.REP)

socketrep.bind(calibration_daemon_searcher_URI)

socketreq = context.socket(zmq.REQ)
socketreq.setsockopt(zmq.LINGER, 0)  #NEW Added by Edwar to flush the queue
socketreq.connect(calibration_daemon_experiment_URI)

socketrep.RCVTIMEO = rcvtimeout  # added timeout to kill rcv if nothing comes
socketreq.RCVTIMEO = rcvtimeout
print(f"done\n\n", flush=True)


# Calibration

In [ ]:
data_path = "/tmp/c3_data/"
optim_name = data_path + "x_gate_calibration"
if not os.path.isdir(data_path):
    os.makedirs(data_path)
if not os.path.isdir(optim_name):
    os.makedirs(optim_name)

In [ ]:
pwd = optim_name+"/"+time.strftime("%d%m%y-%H%M%S", time.localtime())
os.makedirs(pwd)

opts = {
    'CMA_stds': initial_spread,
    'ftarget' : 1e-4,
    'popsize' : 20
}


%matplotlib
plt.rcParams['figure.dpi'] = 100
plt.ion()
plt.show()
fig= plt.figure(figsize=(5, 4))

fridge.calibrate(X_gate, opts)

In [ ]:
X_gate.parameters

### Network shutdown 
After we're done:

In [ ]:
socketrep.unbind(calibration_daemon_searcher_URI)
socketreq.disconnect(calibration_daemon_experiment_URI)